# Ollama + OpenAI + Python

## 1. मॉडेलचे नाव निर्दिष्ट करा

जर तुम्ही "phi3:mini" पेक्षा वेगळे मॉडेल आणले असेल, तर खालील सेलमधील मूल्य बदला. तो व्हेरिएबल संपूर्ण नोटबुकमध्ये कोडमध्ये वापरला जाईल.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Open AI क्लायंट सेटअप करा

सामान्यतः OpenAI क्लायंट OpenAI.com किंवा Azure OpenAI सह मोठ्या भाषा मॉडेल्सशी संवाद साधण्यासाठी वापरला जातो.  
तथापि, तो Ollama सह देखील वापरला जाऊ शकतो, कारण Ollama "http://localhost:11434/v1" येथे OpenAI-सुसंगत एन्डपॉइंट प्रदान करते.


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. चॅट पूर्णता तयार करा

आता आपण OpenAI SDK वापरून संभाषणासाठी प्रतिसाद तयार करू शकतो. ही विनंती मांजरींवर एक हायकू तयार करेल:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. प्रॉम्प्ट इंजिनिअरिंग

भाषा मॉडेलला पाठवलेला पहिला संदेश "सिस्टम संदेश" किंवा "सिस्टम प्रॉम्प्ट" म्हणून ओळखला जातो, आणि तो मॉडेलसाठी एकूण सूचना सेट करतो.  
तुम्ही तुमचा स्वतःचा सानुकूल सिस्टम प्रॉम्प्ट देऊन भाषेच्या मॉडेलला वेगळ्या प्रकारे आउटपुट तयार करण्यासाठी मार्गदर्शन करू शकता.  
खाली दिलेला `SYSTEM_MESSAGE` बदलून तुमच्या आवडत्या प्रसिद्ध चित्रपट/टीव्ही पात्रासारखे उत्तर द्यायला लावा, किंवा इतर सिस्टम प्रॉम्प्टसाठी [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts) मधून प्रेरणा घ्या.

एकदा तुम्ही सिस्टम संदेश सानुकूलित केल्यावर, `USER_MESSAGE` मध्ये पहिला वापरकर्ता प्रश्न द्या.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## ५. काही उदाहरणे

भाषा मॉडेलला मार्गदर्शन करण्याचा आणखी एक मार्ग म्हणजे "काही उदाहरणे" प्रदान करणे, म्हणजे प्रश्न/उत्तरांची एक मालिका जी मॉडेलने कसे प्रतिसाद द्यावे हे दाखवते.

खालील उदाहरणात, भाषा मॉडेलला शिक्षण सहाय्यकासारखे वागवण्याचा प्रयत्न केला आहे. त्यासाठी काही प्रश्न आणि उत्तरांची उदाहरणे दिली आहेत, जी शिक्षण सहाय्यक देऊ शकतो. त्यानंतर विद्यार्थ्याने विचारू शकणाऱ्या प्रश्नासह मॉडेलला प्रॉम्प्ट केले आहे.

हे प्रथम स्वतः करून पहा, आणि नंतर `SYSTEM_MESSAGE`, `EXAMPLES`, आणि `USER_MESSAGE` नवीन परिस्थितीसाठी बदलून पहा.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. माहिती पुनर्प्राप्ती आधारित निर्मिती

RAG (Retrieval Augmented Generation) ही एक तंत्र आहे ज्याद्वारे भाषेचा मॉडेल विशिष्ट क्षेत्रासाठी प्रश्नांची अचूक उत्तरे देतो. यासाठी प्रथम ज्ञान स्रोतातून संबंधित माहिती शोधली जाते आणि नंतर त्या माहितीच्या आधारे उत्तर तयार केले जाते.

आम्ही हायब्रिड कार्सबद्दल डेटा असलेली एक स्थानिक CSV फाइल दिली आहे. खालील कोड CSV फाइल वाचतो, वापरकर्त्याच्या प्रश्नाशी जुळणारी माहिती शोधतो आणि सापडलेल्या माहितीच्या आधारे उत्तर तयार करतो. लक्षात ठेवा की हे मागील उदाहरणांपेक्षा जास्त वेळ घेईल कारण यामध्ये मॉडेलला अधिक डेटा पाठवला जातो. जर तुम्हाला उत्तर डेटा आधारित वाटत नसेल, तर तुम्ही सिस्टम इंजिनिअरिंगचा प्रयत्न करू शकता किंवा इतर मॉडेल्स वापरू शकता. सामान्यतः, RAG मोठ्या मॉडेल्ससह किंवा SLMs च्या फाइन-ट्यून केलेल्या आवृत्त्यांसह अधिक प्रभावी असते.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून निर्माण होणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
